### Joshua Silva

In [49]:
import sqlite3
import pandas as pd
import sklearn.neighbors as knn
import matplotlib.pyplot as plt
import numpy as np
import math
import sklearn.naive_bayes
from scipy import stats
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn import svm, linear_model
import statsmodels.api as sm

# Bulding classifier models for the data
### creating the training and test data sets

In [3]:
#Loading the data
conn = sqlite3.connect('project.db')
c = conn.cursor()
c.execute('SELECT * FROM indication')
indication_data = c.fetchall()
c.execute('SELECT * FROM molecule')
molecule_data = c.fetchall()
conn.close()

In [4]:
# Setting up the dataframe to contain the data
indication_df = pd.DataFrame(indication_data)
indication_df.drop(1, axis = 1, inplace=True)
indication_df.columns = ['id', 'indication']
molecule_df = pd.DataFrame(molecule_data,
                      columns = ['chirality', 'id','acd_logd', 'acd_logp', 'acd_most_apka',
                                 'acd_most_bpka', 'alogp', 'aromatic_rings','full_molformula',
                                 'full_mwt', 'hba', 'hbd', 'heavy_atoms', 'molecular_species',
                                 'qed_weighted'])
joined_df = molecule_df.merge(indication_df, on='id', how='inner')

In [5]:
# spliting into test and train data
training_indexes = np.random.rand(len(joined_df)) < 0.7
training_set = joined_df[training_indexes]
test_set = joined_df[~training_indexes]
print(len(training_set))
print(len(test_set))
print(len(joined_df))

1784
785
2569


# knn classification

In [59]:
knnClassifier = knn.KNeighborsClassifier(n_neighbors=math.floor(math.sqrt(len(training_set))))
# Knn uses numeric feaatures
knnClassifier.fit(training_set.select_dtypes('float64'), training_set['indication'])
predicted = knnClassifier.predict(test_set.select_dtypes('float64'))

In [60]:
# There are many classifications with many having low numbers of records,
# classifers will not be able to classify these
# This is the confusion matrix for each classifier in the test set
print(confusion_matrix(test_set['indication'], predicted))  
print(classification_report(test_set['indication'], predicted))  
#The numbers are the bottom are for the overall performance, see weighted avg

[[0 0 0 ... 0 0 0]
 [0 5 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
                                            precision    recall  f1-score   support

                                 Acidifier       0.00      0.00      0.00         1
                                Adrenergic       0.21      0.56      0.30         9
                    Adrenocortical Steroid       0.00      0.00      0.00         2
                Adrenocortical Suppressant       0.00      0.00      0.00         1
                    Aldosterone Antagonist       0.00      0.00      0.00         1
                                 Alkalizer       0.00      0.00      0.00         2
             Alzheimer's Disease Treatment       0.00      0.00      0.00         1
                                Amino Acid       0.00      0.00      0.00         1
                                 Analgesic       0.07      0.18      0.10        22
                                  Andro

/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [62]:
# Creating a subset of the data where the indications have at least 50 drugs created for them
top_ind_df = indication_df['indication'].value_counts().reset_index()
top_inds = top_ind_df[top_ind_df['indication'] > 50]['index']
top_inds_data = joined_df[joined_df['indication'].isin(top_inds)]
t_subset_indexes = np.random.rand(len(top_inds_data)) < 0.7
training_top_set = top_inds_data[t_subset_indexes]
test_top_set = top_inds_data[~t_subset_indexes]

In [63]:
# KNN on the subset
knnTop = knn.KNeighborsClassifier(n_neighbors=math.floor(math.sqrt(len(training_top_set))))
# Knn uses numeric feaatures
knnTop.fit(training_top_set.select_dtypes('float64'), training_top_set['indication'])
predictedTop = knnTop.predict(test_top_set.select_dtypes('float64'))

In [64]:
# There are many classifications with many having low numbers of records
print(confusion_matrix(test_top_set['indication'], predictedTop))  
print(classification_report(test_top_set['indication'], predictedTop)) 
# Over twice the precision for the weighted average, and recal is increased by about 90%

[[ 5  2  5  7  3  4  4  0  0  1  0  3]
 [ 0 12  0  1  0  0  0  0  1  0  0  0]
 [ 1  0 12  4  0  1  0  0  1  3  0  0]
 [ 0  1  0 51  0  0  2  0  4  2  0  1]
 [ 4  2  0  0  3  6  1  0  0  0  0  0]
 [ 3  0  1  0  1 11  0  0  0  0  0  0]
 [ 2  3  0  9  2  1  6  1  1  3  0  7]
 [ 1  4  0 17  0  0  1  1  6  0  0  2]
 [ 0  2  1  3  1  0  2  0  3  0  0 12]
 [ 0  0  0  2  0  0  0  0  1 13  0  0]
 [ 0  6  1  5  0  0  0  0  1  0  0  0]
 [ 0  0  3  0  2  0  1  0  6  0  0 22]]
                   precision    recall  f1-score   support

        Analgesic       0.31      0.15      0.20        34
       Anesthetic       0.38      0.86      0.52        14
Anti-Inflammatory       0.52      0.55      0.53        22
    Antibacterial       0.52      0.84      0.64        61
   Antidepressant       0.25      0.19      0.21        16
   Antihistaminic       0.48      0.69      0.56        16
 Antihypertensive       0.35      0.17      0.23        35
   Antineoplastic       0.50      0.03      0.06        32

/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Naive Bayes

In [66]:
# Starting with a gausian naive bayes on the data -> as most of the features are continuous
# using those here
gnb = sklearn.naive_bayes.GaussianNB()
naivePred = gnb.fit(training_set.select_dtypes('float64'),
                    training_set['indication']).predict(test_set.select_dtypes('float64'))
gnbTop = sklearn.naive_bayes.GaussianNB()
naiveTopPred = gnbTop.fit(training_top_set.select_dtypes('float64'),
                          training_top_set['indication']).predict(test_top_set.select_dtypes('float64'))

In [68]:
print(confusion_matrix(test_set['indication'], naivePred))  
print(classification_report(test_set['indication'], naivePred))
# For using all data points, the naive bayes set has a higher preciesion but a lower recall

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 5 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
                                                      precision    recall  f1-score   support

                                           Acidifier       0.00      0.00      0.00         1
                                          Adjunct To       0.00      0.00      0.00         0
                                          Adrenergic       0.31      0.56      0.40         9
                              Adrenocortical Steroid       0.06      0.50      0.11         2
                          Adrenocortical Suppressant       0.00      0.00      0.00         1
                              Aldosterone Antagonist       0.14      1.00      0.25         1
                                           Alkalizer       0.00      0.00      0.00         2
                       Alzheimer's Disease Treatment       0.00      0.00      0.00         1
                                 

/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, wa

In [71]:
print(confusion_matrix(test_top_set['indication'], naiveTopPred))  
print(classification_report(test_top_set['indication'], naiveTopPred))
# Compared to kNN the top indication set had similar precision and recall
# The naive bayes classifier had a higher precision, with a lower recall compared to kNN

[[ 7  6  4  2  4  1  5  0  0  0  0  5]
 [ 0  8  0  0  1  0  1  0  0  0  2  2]
 [ 3  1  8  1  1  1  1  0  2  4  0  0]
 [ 3  0  5 27  0  0  8  6  7  0  2  3]
 [ 0  2  0  0 12  0  0  1  0  0  1  0]
 [ 1  0  0  0 13  2  0  0  0  0  0  0]
 [ 2  2  1  3  3  1 15  2  2  1  1  2]
 [ 0  3  0 10  0  1  3  4  2  0  3  6]
 [ 1  0  0  3  1  0  2  2  1  0  0 14]
 [ 0  0  0  0  0  0  0  0  1 15  0  0]
 [ 0  2  0  0  0  0  0  2  0  1  8  0]
 [ 0  0  0  0  2  0  1  0  4  0  0 27]]
                   precision    recall  f1-score   support

        Analgesic       0.41      0.21      0.27        34
       Anesthetic       0.33      0.57      0.42        14
Anti-Inflammatory       0.44      0.36      0.40        22
    Antibacterial       0.59      0.44      0.50        61
   Antidepressant       0.32      0.75      0.45        16
   Antihistaminic       0.33      0.12      0.18        16
 Antihypertensive       0.42      0.43      0.42        35
   Antineoplastic       0.24      0.12      0.16        32

## SVM

In [87]:
svmAll = svm.SVC(gamma='scale').fit(training_set.select_dtypes('float64'),
                                    training_set['indication'])
# Chaining fit with predict did not work ie SVC(),fit().predict()
svmAll = svmAll.predict(test_set.select_dtypes('float64'))
svmTop = svm.SVC(gamma='scale').fit(training_top_set.select_dtypes('float64'),
                                    training_top_set['indication'])
svmTop = svmTop.predict(test_top_set.select_dtypes('float64'))

In [91]:
print(confusion_matrix(test_set['indication'], svmAll))  
print(classification_report(test_set['indication'], svmAll))
# This classifier gave the worst performance

[[0 0 0 ... 0 0 0]
 [0 4 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
                                            precision    recall  f1-score   support

                                 Acidifier       0.00      0.00      0.00         1
                                Adrenergic       0.17      0.44      0.25         9
                    Adrenocortical Steroid       0.00      0.00      0.00         2
                Adrenocortical Suppressant       0.00      0.00      0.00         1
                    Aldosterone Antagonist       0.00      0.00      0.00         1
                                 Alkalizer       0.00      0.00      0.00         2
             Alzheimer's Disease Treatment       0.00      0.00      0.00         1
                                Amino Acid       0.00      0.00      0.00         1
                                 Analgesic       0.04      0.41      0.07        22
                                  Andro

/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [90]:
print(confusion_matrix(test_top_set['indication'], svmTop))  
print(classification_report(test_top_set['indication'], svmTop))
# SVM gave the worst pecision and recall of the methods tested for the top indication set

[[14  0  1 12  4  0  0  0  2  1  0  0]
 [ 4  2  0  1  0  0  0  0  7  0  0  0]
 [ 9  0  4  4  0  0  0  0  2  3  0  0]
 [ 2  0  0 53  0  0  0  0  6  0  0  0]
 [ 5  0  0  0  9  0  0  0  2  0  0  0]
 [ 8  0  0  0  8  0  0  0  0  0  0  0]
 [ 5  0  0 19  1  0  3  0  7  0  0  0]
 [ 0  1  0 18  0  0  0  0 13  0  0  0]
 [ 0  0  0  4  1  0  0  0 19  0  0  0]
 [ 0  0  0  5  0  0  0  0  0 11  0  0]
 [ 0  0  0  6  0  0  0  0  7  0  0  0]
 [ 0  0  0  3  2  0  0  0 29  0  0  0]]
                   precision    recall  f1-score   support

        Analgesic       0.30      0.41      0.35        34
       Anesthetic       0.67      0.14      0.24        14
Anti-Inflammatory       0.80      0.18      0.30        22
    Antibacterial       0.42      0.87      0.57        61
   Antidepressant       0.36      0.56      0.44        16
   Antihistaminic       0.00      0.00      0.00        16
 Antihypertensive       1.00      0.09      0.16        35
   Antineoplastic       0.00      0.00      0.00        32

/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/silvajos/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Models were not good performers for precision and recall
#### This is understandable, data was limited for each classification and this data does not capture drug structue
#### Drug structure is hard to represent and create features for, smiles strings can be used to determine structure, but these were missing for a significant portion of the data, and cannot be imputed

## One of the used features was QED, this is a calculation on how likely a structure is to be a drug
### As classification worked pooorly, using the data for an additional goal of prediction QED.
### This can be done with a linear regression which could not be done for the classification portion

In [7]:
# Get the quantitative variables
linRegTrain_df = training_set.select_dtypes('float64')
linRegTest_df = test_set.select_dtypes('float64')

In [32]:
independent = linRegTrain_df.iloc[:,:-1]
independent = sm.add_constant(indepentent) # The way to add a constant term to model
linReg = sm.OLS(linRegTrain_df.iloc[:,-1], independent).fit()
linReg.summary()
# gives an R^2 of 0.427 with most_bpka, full_mwt, and heavy_atoms being not statistically significant

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           qed_weighted   R-squared:                       0.427
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     132.3
Date:                Sun, 09 Dec 2018   Prob (F-statistic):          2.09e-206
Time:                        17:22:06   Log-Likelihood:                 765.59
No. Observations:                1784   AIC:                            -1509.
Df Residuals:                    1773   BIC:                            -1449.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.8039      0.037     21.668      0.000       0.731       0.877
acd_logd          -0.6204      0.099     -6.290      0.000      -0.814      -0.427
acd_logp           1.0886      0.101     10.734      0.000       0.890       1.288
acd_most_apka      0.2675      0.032      8.407      0.000       0.205       0.330
acd_most_bpka     -0.0204      0.028     -0.719      0.472      -0.076       0.035
alogp             -0.8393      0.097     -8.613      0.000      -1.030      -0.648
aromatic_rings     0.4017      0.038     10.524      0.000       0.327       0.477
full_mwt          -0.1110      0.109     -1.019      0.309      -0.325       0.103
hba               -0.6233      0.077     -8.102      0.000      -0.774      -0.472
hbd               -0.5603      0.037    -15.170      0.000      -0.633      -0.488
heavy_atoms       -0.0699      0.081     -0.863      0.388      -0.229       0.089
==============================================================================
Omnibus:                       55.394   Durbin-Watson:                   1.653
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.811
Skew:                          -0.440   Prob(JB):                     1.03e-13
Kurtosis:                       3.171   Cond. No.                         59.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
# Droping the non statistically significant independent variables 
independent = linRegTrain_df.iloc[:,:-1]
independent = sm.add_constant(indepentent) # The way to add a constant term to model
linReg = sm.OLS(linRegTrain_df.iloc[:,-1],
                independent.drop(columns = ['acd_most_bpka', 'full_mwt', 'heavy_atoms'])).fit()
linReg.summary()
# gives an R^2 of 0.426 all independent variables matter

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           qed_weighted   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     188.7
Date:                Sun, 09 Dec 2018   Prob (F-statistic):          3.39e-209
Time:                        17:28:08   Log-Likelihood:                 764.23
No. Observations:                1784   AIC:                            -1512.
Df Residuals:                    1776   BIC:                            -1469.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.8214      0.029     27.920      0.000       0.764       0.879
acd_logd          -0.5943      0.093     -6.392      0.000      -0.777      -0.412
acd_logp           1.0719      0.094     11.344      0.000       0.887       1.257
acd_most_apka      0.2583      0.030      8.625      0.000       0.200       0.317
alogp             -0.9246      0.062    -14.932      0.000      -1.046      -0.803
aromatic_rings     0.4061      0.038     10.748      0.000       0.332       0.480
hba               -0.6866      0.037    -18.352      0.000      -0.760      -0.613
hbd               -0.5765      0.035    -16.435      0.000      -0.645      -0.508
==============================================================================
Omnibus:                       51.502   Durbin-Watson:                   1.653
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.263
Skew:                          -0.424   Prob(JB):                     9.99e-13
Kurtosis:                       3.161   Cond. No.                         49.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
# Not including a constant term gives a better model
linReg = sm.OLS(linRegTrain_df.iloc[:,-1], linRegTrain_df.iloc[:,:-1]).fit()
linReg.summary()
# R-squared if 0.926
# full_mwt is again not a statistically significant independent variable

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           qed_weighted   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     2233.
Date:                Sun, 09 Dec 2018   Prob (F-statistic):               0.00
Time:                        17:24:45   Log-Likelihood:                 556.04
No. Observations:                1784   AIC:                            -1092.
Df Residuals:                    1774   BIC:                            -1037.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
acd_logd          -1.0561      0.109     -9.729      0.000      -1.269      -0.843
acd_logp           1.4452      0.113     12.844      0.000       1.224       1.666
acd_most_apka      0.5617      0.032     17.365      0.000       0.498       0.625
acd_most_bpka      0.0932      0.031      2.968      0.003       0.032       0.155
alogp              0.5651      0.082      6.908      0.000       0.405       0.726
aromatic_rings     0.3418      0.043      7.986      0.000       0.258       0.426
full_mwt          -0.0767      0.122     -0.626      0.532      -0.317       0.164
hba                0.4197      0.067      6.219      0.000       0.287       0.552
hbd               -0.3051      0.039     -7.750      0.000      -0.382      -0.228
heavy_atoms       -1.0964      0.074    -14.851      0.000      -1.241      -0.952
==============================================================================
Omnibus:                       61.855   Durbin-Watson:                   1.644
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               79.961
Skew:                          -0.374   Prob(JB):                     4.33e-18
Kurtosis:                       3.718   Cond. No.                         45.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
# Droping the full_mwt independent feature
linReg = sm.OLS(linRegTrain_df.iloc[:,-1], linRegTrain_df.iloc[:,:-1].drop(columns = 'full_mwt')).fit()
linReg.summary()
# Same R-squared with less overfitting

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           qed_weighted   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     2481.
Date:                Sun, 09 Dec 2018   Prob (F-statistic):               0.00
Time:                        17:33:48   Log-Likelihood:                 555.84
No. Observations:                1784   AIC:                            -1094.
Df Residuals:                    1775   BIC:                            -1044.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
acd_logd          -1.0565      0.109     -9.735      0.000      -1.269      -0.844
acd_logp           1.4470      0.112     12.866      0.000       1.226       1.668
acd_most_apka      0.5621      0.032     17.385      0.000       0.499       0.626
acd_most_bpka      0.0931      0.031      2.963      0.003       0.031       0.155
alogp              0.5616      0.082      6.882      0.000       0.402       0.722
aromatic_rings     0.3418      0.043      7.987      0.000       0.258       0.426
hba                0.4184      0.067      6.204      0.000       0.286       0.551
hbd               -0.3062      0.039     -7.790      0.000      -0.383      -0.229
heavy_atoms       -1.1045      0.073    -15.192      0.000      -1.247      -0.962
==============================================================================
Omnibus:                       60.043   Durbin-Watson:                   1.642
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               77.518
Skew:                          -0.367   Prob(JB):                     1.47e-17
Kurtosis:                       3.710   Cond. No.                         45.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
# Using the latest model to predict terms:
predictions = linReg.predict(linRegTest_df.iloc[:,:-1].drop(columns = 'full_mwt'))
print(mean_squared_error(predictions,linRegTest_df.iloc[:,-1]))
# The r-squared was -0.68, this is not predictive of the QED for the test set
# Using average QED would be more useful as r-squared is negative
print(r2_score(predictions, linRegTest_df.iloc[:,-1]))

0.036284707776516587
-0.6841924895743399
